In [1]:
import os
os.chdir("..")

In [2]:
import os

import wandb
from omegaconf import OmegaConf
from tqdm import tqdm

In [3]:
import pandas as pd

In [4]:
from helpers.evaluators import load_checkpoint_and_cfg

In [4]:
CKPT_DIR = "/scratch/izar/cizinsky/garment-completion/artifacts"

In [5]:
!mkdir -p {CKPT_DIR}

In [5]:
tag = "part1"

In [6]:
api = wandb.Api()

runs = api.runs("ludekcizinsky/pbr-generation")

tagged_runs = [run for run in runs if tag in run.tags]

for i, run in enumerate(tagged_runs):
    print(f"{i}/{len(tagged_runs)} {run.id} | {run.name} | tags: {run.tags} | {run.summary.get('final_eval/lpips', 'N/A')}")

0/20 4rc12jo5 | eager-energy-111 | tags: ['ddim', 'part1'] | 0.5848165154457092
1/20 4dro7oe6 | honest-bird-113 | tags: ['lr', 'part1'] | 0.8321104645729065
2/20 ha6wswub | morning-microwave-118 | tags: ['lora', 'lr', 'part1'] | 0.7591138482093811
3/20 5ajtw75b | sandy-capybara-119 | tags: ['ddim', 'part1'] | 0.7796162366867065
4/20 6ei613pt | valiant-resonance-120 | tags: ['lr', 'part1'] | 0.44604727625846863
5/20 b835hnfq | cosmic-cosmos-123 | tags: ['lora', 'lr', 'part1'] | 0.6090513467788696
6/20 y5appjkm | stellar-feather-136 | tags: ['part1', 'pix2pix', 'scratch'] | 0.8173788189888
7/20 w5daifhx | fast-universe-159 | tags: ['baseline', 'part1'] | 0.6893279552459717
8/20 fstkreeo | twilight-sponge-160 | tags: ['ddim', 'part1'] | 0.6316825151443481
9/20 2afus61b | grateful-terrain-163 | tags: ['ema', 'part1'] | 0.6981697082519531
10/20 dg766dgh | glowing-disco-164 | tags: ['lr-scheduler', 'part1'] | 0.6885690689086914
11/20 kv3lwz0e | fanciful-wood-176 | tags: ['part1', 'pix2pix', 

In [7]:
id2run = {run.id: run for run in tagged_runs}

In [11]:
cfgs = dict()

for run in tqdm(tagged_runs):
    try:
        ckpt, cfg = load_checkpoint_and_cfg(run.name)
        cfgs[run.id] = cfg
    except Exception as e:
        # save empty dict config
        cfgs[run.id] = OmegaConf.create({})

100%|██████████| 20/20 [03:46<00:00, 11.32s/it]


In [12]:
save_dir = "saved_configs"
os.makedirs(save_dir, exist_ok=True)

for run_id, cfg in cfgs.items():
    OmegaConf.save(cfg, os.path.join(save_dir, f"{run_id}.yaml"))

In [8]:
load_dir = "notebooks/saved_configs"
run_configs = {}

for filename in os.listdir(load_dir):
    if filename.endswith(".yaml"):
        run_id = filename.replace(".yaml", "")
        cfg = OmegaConf.load(os.path.join(load_dir, filename))
        run_configs[run_id] = cfg

In [9]:
rows = []
for run_id, cfg in run_configs.items():
    if len(cfg) == 0:
        continue
    ddim = cfg.optim.ddim_loss_weight
    lr = cfg.optim.lr
    bsz = cfg.data.batch_size

    if hasattr(cfg.data, "trn_debug_size"):
        trn_size = cfg.data.trn_debug_size
    else:
        trn_size = 27000
    if trn_size == -1:
        trn_size = 27000

    run = id2run[run_id]
    if hasattr(cfg.model, "train_with_lora"):
        is_lora = cfg.model.train_with_lora
    elif run.id in ["7r6s0noz", "5txmj6i1", "ha6wswub", "b835hnfq"]:
        is_lora = True
    else:
        is_lora = False

    if run.id == "dg766dgh":
        use_cos_sche = True
    else:
        use_cos_sche = False
    
    if run.id == "2afus61b":
        use_ema = True
    else:
        use_ema = False

    if run.id == "y5appjkm":
        from_scratch = True
    else:
        from_scratch = False


    if run.id in ["uq7v05g2", "xwexoq8d"]:
        is_inpainting = True
    else:
        is_inpainting = False

    if is_inpainting:
        base_model = "sd-inpainting"
    elif is_lora:
        base_model = "pix2pix"
    else:
        base_model = "sd"

    if run.id == "6ei613pt":
        trn_time_hrs = 60
    else:
        trn_time_hrs = 30

    lpips = run.summary.get("final_eval/lpips", "N/A")
    ssim = run.summary.get("final_eval/ssim", "N/A")
    psnr = run.summary.get("final_eval/psnr", "N/A")

    rows.append({
        "run_id": run_id,
        "ddim": ddim,
        "lr": lr,
        "bsz": bsz,
        "trn_size": trn_size,
        "is_lora": is_lora,
        "use_cos_sche": use_cos_sche,
        "use_ema": use_ema,
        "from_scratch": from_scratch,
        "is_inpainting": is_inpainting,
        "base_model": base_model,
        "trn_time_hrs": trn_time_hrs,
        "lpips": lpips,
        "ssim": ssim,
        "psnr": psnr,
    })





In [10]:
df = pd.DataFrame(rows)

# sort by is_lora, trn_size and by trn_time_hrs
df = df.sort_values(by=["is_lora", "trn_size", "trn_time_hrs"], ascending=False)

df

,run_id,ddim,lr,bsz,trn_size,is_lora,use_cos_sche,use_ema,from_scratch,is_inpainting,base_model,trn_time_hrs,lpips,ssim,psnr
1,7r6s0noz,0.50,0.005000,40,27000,True,False,False,False,False,pix2pix,30,N/A,N/A,N/A
7,b835hnfq,0.50,0.000100,20,27000,True,False,False,False,False,pix2pix,30,0.609051,0.120764,12.15182
11,5txmj6i1,0.50,0.000100,40,27000,True,False,False,False,False,pix2pix,30,N/A,N/A,N/A
13,ha6wswub,0.50,0.000010,20,27000,True,False,False,False,False,pix2pix,30,0.759114,0.100942,10.62117
15,uq7v05g2,0.75,0.000100,20,27000,True,False,False,False,True,sd-inpainting,30,0.960359,0.126441,9.246959
16,6ei613pt,0.50,0.000100,20,27000,False,False,False,False,False,sd,60,0.446047,0.171383,13.036096
0,dg766dgh,0.50,0.000010,20,27000,False,True,False,False,False,sd,30,0.688569,0.122153,10.697735
2,2afus61b,0.50,0.000010,10,27000,False,False,True,False,False,sd,30,0.69817,0.119585,11.396476
3,4dro7oe6,0.50,0.000003,20,27000,False,False,False,False,False,sd,30,0.83211,0.090923,10.220429
4,y5appjkm,0.50,0.000010,20,27000,False,False,False,True,False,sd,30,0.817379,0.022002,7.481595


In [56]:
df_hyper_params = df [(~df["is_lora"]) & (~df["from_scratch"]) & (~df["is_inpainting"]) & (df["trn_size"] == 27000)].drop(columns=["is_inpainting", "from_scratch", "is_lora", "bsz", "trn_size", "base_model", "trn_time_hrs", "run_id"]) 

# sort by lpips (ascending), ssim (descending), psnr (descending)
df_hyper_params = df_hyper_params.sort_values(by=["lpips", "ssim", "psnr"], ascending=[True, False, False])

df_hyper_params.to_csv("hyper_params.csv", index=False)

In [13]:
df_architecture = df[(df["is_lora"]) | (df["from_scratch"]) | (df["is_inpainting"])].drop(columns=["trn_size", "trn_time_hrs", "use_ema", "use_cos_sche", "run_id"])

# sort by lpips (ascending), ssim (descending), psnr (descending)
df_architecture = df_architecture.sort_values(by=["lpips", "ssim", "psnr"], ascending=[True, False, False])

df_architecture.to_csv("architecture.csv", index=False)

In [70]:
df_scale = df[(df["trn_size"] < 27000) | (df["trn_time_hrs"] == 60)].drop(columns=["use_ema", "use_cos_sche", "is_inpainting", "from_scratch", "is_lora", "base_model", "bsz", "run_id", "lr", "ddim"])

# sort by lpips (ascending), ssim (descending), psnr (descending)
df_scale = df_scale.sort_values(by=["lpips", "ssim", "psnr"], ascending=[True, False, False])

df_scale.to_csv("scale.csv", index=False)